In [ ]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 44.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=675a1ac6910db9411200d5ae82acf75447d6d31d9edbee9e08228f2e15a701aa
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

In [595]:
session = SparkSession.builder.appName("Mycode").master("local").getOrCreate()
data = session.read.csv("Admission_Prediction.csv", header = True, inferSchema=True)
data.show()

+---+----------+---------+-----------+-----------------+---+---+----+--------+---------------+
|_c0|Serial No.|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---+----------+---------+-----------+-----------------+---+---+----+--------+---------------+
|  0|         1|    337.0|      118.0|              4.0|4.5|4.5|9.65|       1|           0.92|
|  1|         2|    324.0|      107.0|              4.0|4.0|4.5|8.87|       1|           0.76|
|  3|         4|    322.0|      110.0|              3.0|3.5|2.5|8.67|       1|            0.8|
|  4|         5|    314.0|      103.0|              2.0|2.0|3.0|8.21|       0|           0.65|
|  5|         6|    330.0|      115.0|              5.0|4.5|3.0|9.34|       1|            0.9|
|  7|         8|    308.0|      101.0|              2.0|3.0|4.0| 7.9|       0|           0.68|
|  8|         9|    302.0|      102.0|              1.0|2.0|1.5| 8.0|       0|            0.5|
|  9|        10|    323.0|      108.0|            

In [596]:
data.describe()

DataFrame[summary: string, _c0: string, Serial No.: string, GRE Score: string, TOEFL Score: string, University Rating: string, SOP: string, LOR: string, CGPA: string, Research: string, Chance of Admit: string]

In [597]:
data.dtypes

[('_c0', 'int'),
 ('Serial No.', 'int'),
 ('GRE Score', 'double'),
 ('TOEFL Score', 'double'),
 ('University Rating', 'double'),
 ('SOP', 'double'),
 ('LOR', 'double'),
 ('CGPA', 'double'),
 ('Research', 'int'),
 ('Chance of Admit', 'double')]

In [598]:
data.columns

['_c0',
 'Serial No.',
 'GRE Score',
 'TOEFL Score',
 'University Rating',
 'SOP',
 'LOR',
 'CGPA',
 'Research',
 'Chance of Admit']

In [599]:
data.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Serial No.: integer (nullable = true)
 |-- GRE Score: double (nullable = true)
 |-- TOEFL Score: double (nullable = true)
 |-- University Rating: double (nullable = true)
 |-- SOP: double (nullable = true)
 |-- LOR: double (nullable = true)
 |-- CGPA: double (nullable = true)
 |-- Research: integer (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [600]:
data.count()

460

In [601]:
data.show()

+---+----------+---------+-----------+-----------------+---+---+----+--------+---------------+
|_c0|Serial No.|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---+----------+---------+-----------+-----------------+---+---+----+--------+---------------+
|  0|         1|    337.0|      118.0|              4.0|4.5|4.5|9.65|       1|           0.92|
|  1|         2|    324.0|      107.0|              4.0|4.0|4.5|8.87|       1|           0.76|
|  3|         4|    322.0|      110.0|              3.0|3.5|2.5|8.67|       1|            0.8|
|  4|         5|    314.0|      103.0|              2.0|2.0|3.0|8.21|       0|           0.65|
|  5|         6|    330.0|      115.0|              5.0|4.5|3.0|9.34|       1|            0.9|
|  7|         8|    308.0|      101.0|              2.0|3.0|4.0| 7.9|       0|           0.68|
|  8|         9|    302.0|      102.0|              1.0|2.0|1.5| 8.0|       0|            0.5|
|  9|        10|    323.0|      108.0|            

In [602]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder

In [603]:
vec_ass = VectorAssembler(inputCols=[
'_c0',
 'GRE Score',
 'TOEFL Score',
 'University Rating',
 'SOP',
 'LOR',
 'CGPA',
 'Research'], outputCol="allfeatures")

In [604]:
from pyspark.ml.regression import LinearRegression
linear = LinearRegression(featuresCol="allfeatures", labelCol="Chance of Admit")

In [605]:
newdata = vec_ass.transform(data)

In [606]:
from pyspark.ml import Pipeline

In [607]:
training, test = newdata.randomSplit([.8, .2])

In [608]:
model = linear.fit(training)

In [609]:
result = model.transform(test)

In [610]:
result.show()

+---+----------+---------+-----------+-----------------+---+---+----+--------+---------------+--------------------+-------------------+
|_c0|Serial No.|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|         allfeatures|         prediction|
+---+----------+---------+-----------+-----------------+---+---+----+--------+---------------+--------------------+-------------------+
|  4|         5|    314.0|      103.0|              2.0|2.0|3.0|8.21|       0|           0.65|[4.0,314.0,103.0,...| 0.6066976908422705|
| 16|        17|    317.0|      107.0|              3.0|4.0|3.0| 8.7|       0|           0.66|[16.0,317.0,107.0...| 0.6962740877334241|
| 33|        34|    340.0|      114.0|              5.0|4.0|4.0| 9.6|       1|            0.9|[33.0,340.0,114.0...| 0.9161542745634963|
| 38|        39|    304.0|      105.0|              1.0|3.0|1.5| 7.5|       0|           0.52|[38.0,304.0,105.0...|0.49835997315382397|
| 42|        43|    313.0|      107.0|          

In [611]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(predictionCol="prediction", labelCol='Chance of Admit', metricName="r2")
eval.evaluate(result)

0.8490299989697287

In [612]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(predictionCol="prediction", labelCol='Chance of Admit', metricName="rmse")
eval.evaluate(result)

0.06430619732721732

In [613]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(predictionCol="prediction", labelCol='Chance of Admit', metricName="mse")
eval.evaluate(result)

0.0041352870146870125

In [614]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(predictionCol="prediction", labelCol='Chance of Admit', metricName="mae")
eval.evaluate(result)

0.048282776815045